In [ ]:
from search import *
from model_loader.config import *
from save_utils import *
import time

class CarManualQA:
    def __init__(self, generation_loader, data_folder="./data/split_file", prompt_path="./prompts/ko/generation/cogito-qwen/generation_prompt2.txt", result_path="./result"):
        self.data_folder = data_folder
        self.result_path = result_path
        self.searcher = HybridSearcher(embedding_loader, chunk_size=200, chunk_overlap=50)
        self.loader = generation_loader
        self.prompt_path = prompt_path
        self.prompt_template = self._load_prompt(prompt_path)
        self.category_to_file = {
            '1': "1_키.txt",  # 1~13, 73
            '2': "2_중앙_자동_잠금장치.txt",  # 14~18
            '4': "3_도어.txt",  # 19~22
            '5': "4_차량안전.txt",  # 22~25
            '6': "5_미러.txt",  # 26~29
            '7': "6_유리창_및_루프.txt",  # 40~46
            '9': "7_헤드레스트_시트.txt",  # 47~54, 74~78
            '11': "8_안전벨트.txt",  # 54~59
            '12': "9_에어백.txt",  # 60~68
            '13': "10_어린이_시트.txt",  # 68~72
            '14': "11_운전석.txt",  # 79~90
            '15': "12_하이패스.txt"  # 29~40
        }

    def generate_response(self, query, category, top_n=5, alpha=0.5):
        time_logs = {}
        total_start = time.time()
        
        # 단계 1: 문서 로드
        t_start = time.time()
        filename = self.category_to_file[category]
        file_path = os.path.join(self.data_folder, filename)
        self.searcher.load_document(file_path)
        time_logs['1_문서_로드'] = time.time() - t_start
        print(f"1. 문서 로드: {time_logs['1_문서_로드']:.2f}초")
        
        # 단계 2: 검색 실행
        t_start = time.time()
        search_results = self.searcher.search(query, top_n=top_n, alpha=alpha)
        time_logs['2_검색_실행'] = time.time() - t_start
        print(f"2. 검색 실행: {time_logs['2_검색_실행']:.2f}초")
        
        # 단계 3: 컨텍스트 준비
        t_start = time.time()
        context = "\n\n".join([result["chunk"] for result in search_results])
        prompt = self.prompt_template.format(context=context, context_qa="", query=query)
        time_logs['3_컨텍스트_준비'] = time.time() - t_start
        print(f"3. 컨텍스트 준비: {time_logs['3_컨텍스트_준비']:.2f}초")
        
        # 단계 4: 토크나이징
        t_start = time.time()
        if hasattr(self.loader, "tokenizer"):
            tokenizer = self.loader.tokenizer
            model = self.loader.model
            # input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
            input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True)
            input_ids = input_ids.repeat(2, 1).to(model.device)  # 배치 크기 2로 설정
            attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
            time_logs['4_토크나이징'] = time.time() - t_start
            print(f"4. 토크나이징: {time_logs['4_토크나이징']:.2f}초")
            
            # 단계 5: 모델 생성 (가장 시간이 많이 걸릴 것으로 예상)
            t_start = time.time()
            t_start = time.time()
            output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=200,
                temperature=0.3,
                do_sample=True,
                top_p=0.85,
                repetition_penalty=1.2,
                early_stopping=True,
                num_beams=3,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            time_logs['5_모델_생성'] = time.time() - t_start
            print(f"5. 모델 생성: {time_logs['5_모델_생성']:.2f}초")
            
            # 단계 6: 디코딩
            t_start = time.time()
            generated_ids = output[0][input_ids.shape[-1]:]
            raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            time_logs['6_디코딩'] = time.time() - t_start
            print(f"6. 디코딩: {time_logs['6_디코딩']:.2f}초")
        else:
            # API 모델 사용 시
            t_start = time.time()
            raw_answer = self.loader.generate(prompt)
            time_logs['4_API_생성'] = time.time() - t_start
            print(f"4. API 생성: {time_logs['4_API_생성']:.2f}초")

        # 단계 7: 후처리
        t_start = time.time()
        final_answer = self._extract_answer_content(raw_answer)
        final_answer = self._remove_chinese_characters(final_answer)
        time_logs['7_후처리'] = time.time() - t_start
        print(f"7. 후처리: {time_logs['7_후처리']:.2f}초")
        
        # 단계 8: 결과 저장
        t_start = time.time()
        response = {
            "답변": raw_answer,
            "후처리": final_answer,
            "문서 일부": context,
            "question_en": query,
            "answer_en": raw_answer,
            "검색_결과": search_results,
            "시간_로그": time_logs
        }

        try:
            result_path = getattr(self, 'result_path', '../result')
            alpha_str = f"{alpha:.1f}"
            alpha_folder = os.path.join(result_path, f"alpha_{alpha_str}")
            os.makedirs(alpha_folder, exist_ok=True)
            
            save_response_to_file(
                query=query,
                answer=response["답변"],
                final_answer=response["후처리"],
                context=response["문서 일부"],
                folder=alpha_folder
            )
        except Exception as e:
            print(f"결과 저장 중 오류 발생: {e}")
        
        time_logs['8_결과_저장'] = time.time() - t_start
        print(f"8. 결과 저장: {time_logs['8_결과_저장']:.2f}초")
        
        # 총 소요 시간
        total_time = time.time() - total_start
        time_logs['총_소요_시간'] = total_time
        print(f"총 소요 시간: {total_time:.2f}초")
        
        # 각 단계별 비율 출력
        print("\n각 단계별 시간 비율:")
        for key, value in time_logs.items():
            if key != '총_소요_시간':
                percentage = (value / total_time) * 100
                print(f"  {key}: {percentage:.1f}%")
        
        return response
    
    def _load_prompt(self, path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"[ERROR] 프롬프트 로드 실패: {e}")
            raise
    
    def _extract_answer_content(self, text):
        pattern = r"(?:<\|?|<|)?\|?answer\|?(?:\|?>|>)?(.*?)(?:<\|?|<|)?\|?endanswer\|?(?:\|?>|>)?"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else text.strip()
    
    def _remove_chinese_characters(self, text):
        return re.sub(r'[\u4E00-\u9FFF]', '', text)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

: 

In [28]:
import time
from model_loader.config import generation_loader

# CarManualQA 시스템 초기화
qa_system = CarManualQA(generation_loader)

# 테스트할 질문과 카테고리 선택
question = "애가 문 못열게 하는거 어케함?"
category = '2'
alpha = 0.5

print(f"질문: {question}")
print(f"카테고리: {category}")
print(f"알파: {alpha}")
print("-" * 50)

# generate_response 함수 실행 (내부에서 시간 측정 및 출력됨)
result = qa_system.generate_response(question, category, alpha=alpha)

# 답변 출력
print("\n생성된 답변:")
print(result["후처리"])

질문: 애가 문 못열게 하는거 어케함?
카테고리: 2
알파: 0.5
--------------------------------------------------
1. 문서 로드: 0.11초
2. 검색 실행: 0.04초
3. 컨텍스트 준비: 0.00초
4. 토크나이징: 0.01초
5. 모델 생성: 167.49초
6. 디코딩: 0.00초
7. 후처리: 0.00초
8. 결과 저장: 0.00초
총 소요 시간: 167.65초

각 단계별 시간 비율:
  1_문서_로드: 0.1%
  2_검색_실행: 0.0%
  3_컨텍스트_준비: 0.0%
  4_토크나이징: 0.0%
  5_모델_생성: 99.9%
  6_디코딩: 0.0%
  7_후처리: 0.0%
  8_결과_저장: 0.0%

생성된 답변:
참고 이미지 : 
![어린이 안전 잠금 장치](./image/image_18_1.png)

18페이지에 따르면, 뒤쪽 도어의 어린이 안전 잠금 장치를 위쪽으로 올려 잠그면 도어는 실내에서 열 수 없습니다. 기능을 해제하려면 실외 열림 레버로 뒤쪽 도어를 열고 어린이 안전 잠금 장치를 해제 방향으로 내리면 됩니다.

주의: 어린이 안전 도어 잠금 장치가 LOCK 위치에 있을 때에는 실내 도어 손잡이를 당기지 마십시오. 실내 도어 손잡이가 손상될 수 있습니다.


In [22]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

_translation_model = None
_translation_tokenizer = None

MODEL_NAME = "../model/translate/facebook-m2m100"

def get_translation_model():
    global _translation_model, _translation_tokenizer
    if _translation_model is None or _translation_tokenizer is None:
        _translation_tokenizer = M2M100Tokenizer.from_pretrained(MODEL_NAME)
        _translation_model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME)
    return _translation_tokenizer, _translation_model

def translate_en_to_ko(text):
    tokenizer, model = get_translation_model()
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt")
    generated = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ko"))
    return tokenizer.decode(generated[0], skip_special_tokens=True)

In [23]:
import numpy as np
import re
import os
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from model_loader.config import embedding_loader

class HybridSearcher :
    def __init__(self, embedding_loader, chunk_size=200, chunk_overlap=50) :
        self.embedding_model = embedding_loader
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.chunks = None
        self.chunk_metadata = None
        self.bm25_index = None
        self.vector_index = None

    def load_document(self, file_path):
        """문서 로드 및 청크 분할"""
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        # 문서를 청크로 분할
        self.chunks, self.chunk_metadata = self._split_into_chunks_with_metadata(content)
        
        # BM25 인덱스 생성
        tokenized_chunks = [self._simple_tokenize(chunk) for chunk in self.chunks]
        self.bm25_index = BM25Okapi(tokenized_chunks)
        
        # 벡터 임베딩 생성 및 인덱스 구축
        self.vector_index = self.embedding_model.encode(self.chunks)
        
        return len(self.chunks)
    
    def search(self, query, top_n=5, alpha=0.5):
        """하이브리드 검색 수행"""
        if self.chunks is None or self.bm25_index is None or self.vector_index is None:
            raise ValueError("문서가 로드되지 않았습니다. load_document()를 먼저 호출하세요.")
        
        # BM25 검색 수행
        bm25_scores = self.bm25_index.get_scores(self._simple_tokenize(query))
        
        # 벡터 검색 수행
        query_embedding = self.embedding_model.encode([query])[0]
        vector_scores = cosine_similarity([query_embedding], self.vector_index)[0]
        
        # 검색 결과 결합
        combined_scores = self._combine_scores(bm25_scores, vector_scores, alpha)
        
        # 상위 N개 결과 반환
        top_indices = np.argsort(combined_scores)[-top_n:][::-1]
        results = [
            {
                "chunk": self.chunks[i],
                "score": combined_scores[i],
                "bm25_score": bm25_scores[i],
                "vector_score": vector_scores[i],
                "index": i,
                "page" : self.chunk_metadata[i]["page"]
            }
            for i in top_indices
        ]
        
        return results

    def _split_into_chunks_with_metadata(self, text):
        """텍스트를 청크로 분할하고 페이지 정보를 메타데이터로 유지하는 함수"""
        chunks = []
        chunk_metadata = []
        
        # 페이지 패턴 정규식 (####으로 시작하는 페이지 헤더)
        page_pattern = re.compile(r'####\s*(\d+)페이지')
        
        # 텍스트를 줄 단위로 처리
        lines = text.split('\n')
        current_page = "unknown"
        current_chunk = ""
        
        for line in lines:
            # 페이지 헤더 확인
            page_match = page_pattern.match(line)
            
            if page_match:
                # 새 페이지 시작
                # 현재 청크가 있으면 저장
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                    chunk_metadata.append({"page": current_page})
                    current_chunk = ""
                
                # 새 페이지 번호 설정
                current_page = page_match.group(1)
                continue
            
            # 현재 청크에 라인 추가
            current_chunk += line + "\n"
            
            # 청크 크기 확인
            if len(current_chunk) >= self.chunk_size:
                chunks.append(current_chunk.strip())
                chunk_metadata.append({"page": current_page})
                current_chunk = ""  # 새 청크 시작
        
        # 마지막 청크 처리
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
            chunk_metadata.append({"page": current_page})
        
        # 너무 작은 청크 결합 (메타데이터 유지)
        i = 0
        while i < len(chunks) - 1:
            if len(chunks[i]) + len(chunks[i+1]) < self.chunk_size:
                # 같은 페이지인 경우에만 결합
                if chunk_metadata[i]["page"] == chunk_metadata[i+1]["page"]:
                    chunks[i] = chunks[i] + "\n\n" + chunks[i+1]
                    chunks.pop(i+1)
                    chunk_metadata.pop(i+1)
                else:
                    i += 1
            else:
                i += 1
        
        return chunks, chunk_metadata
    
    def _simple_tokenize(self, text):
        """텍스트를 간단히 토크나이징하는 함수"""
        return re.findall(r'\w+', text.lower())
    
    def _combine_scores(self, bm25_scores, vector_scores, alpha=0.5):
        """BM25와 벡터 검색 점수를 결합"""
        # 점수 정규화
        if np.max(bm25_scores) > 0:
            bm25_scores = bm25_scores / np.max(bm25_scores)
        if np.max(vector_scores) > 0:
            vector_scores = vector_scores / np.max(vector_scores)
        
        # 가중 평균 계산
        combined = alpha * bm25_scores + (1 - alpha) * vector_scores
        return combined
    
    def get_chunks_with_page_info(self, indices=None) :
        if indices is None :
            return [(chunk, self.chunk_metadata[i]["page"]) for i, chunk in enumerate(self.chunks)]
        else :
            return [(self.chunks[i], self.chunk_metadata[i]["page"]) for i in indices if i < len(self.chunks)]

In [26]:
from model_loader.config import *
from save_utils import *

class CarManualQA:
    def __init__(self, generation_loader, data_folder="./data/split_file", 
                 ko_prompt_path="./prompts/ko/generation/gemma3/generation_prompt.txt", 
                 en_prompt_path="./prompts/ko/generation/gemma3/generation_prompt.txt", 
                 result_path="./result/5월9일/gemma3_full"):
        self.data_folder = data_folder
        self.result_path = result_path
        self.searcher = HybridSearcher(embedding_loader, chunk_size=200, chunk_overlap=50)
        self.loader = generation_loader
        self.ko_prompt_path = ko_prompt_path
        self.en_prompt_path = en_prompt_path
        self.ko_prompt_template = self._load_prompt(ko_prompt_path)
        self.en_prompt_template = self._load_prompt(en_prompt_path)
        self.category_to_file = {
            '1': "1_키.txt",  # 1~13, 73
            '2': "2_중앙_자동_잠금장치.txt",  # 14~18
            '4': "3_도어.txt",  # 19~22
            '5': "4_차량안전.txt",  # 22~25
            '6': "5_미러.txt",  # 26~29
            '7': "6_유리창_및_루프.txt",  # 40~46
            '9': "7_헤드레스트_시트.txt",  # 47~54, 74~78
            '11': "8_안전벨트.txt",  # 54~59
            '12': "9_에어백.txt",  # 60~68
            '13': "10_어린이_시트.txt",  # 68~72
            '14': "11_운전석.txt",  # 79~90
            '15': "12_하이패스.txt",  # 29~40,
            "16": "full.txt"
        }

    def generate_response(self, query, category, top_n=5, alpha=0.5, language="ko"):
        # 검색 파트
        filename = self.category_to_file[category]
        file_path = os.path.join(self.data_folder, filename)

        self.searcher.load_document(file_path)
        search_results = self.searcher.search(query, top_n=top_n, alpha=alpha)
        
        context = "\n\n".join([f"#### {result['page']}페이지\n{result['chunk']}" for result in search_results])
        
        # 응답 생성 파트
        if language.lower() == "ko" :
            print("번역 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            prompt_template = self.ko_prompt_template
        else :
            print("번역 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO")
            prompt_template = self.en_prompt_template
        
        prompt = prompt_template.format(context=context, context_qa="", query=query)

        if hasattr(self.loader, "tokenizer"):
            tokenizer = self.loader.tokenizer
            model = self.loader.model

            # 인풋 토크나이즈
            input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
            attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
            
            # 텍스트 생성
            output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=400,
                temperature=0.3,
                do_sample=False,
                top_p=0.85,
                repetition_penalty=1.2,
                early_stopping=True,
                num_beams=3,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            generated_ids = output[0][input_ids.shape[-1]:]
            raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        else:
            raw_answer = self.loader.generate(prompt)

        final_answer = self._extract_answer_content(raw_answer)
        final_answer = self._remove_chinese_characters(final_answer)
        
        translated_answer = None
        if language.lower != "ko" :
            try :
                translated_answer = translate_en_to_ko(final_answer)
            except Exception as e :
                print(f"번역 중 오류 발생 : {e}")
                translated_answer = f"[번역 실패] {final_answer}"

        # 결과 준비
        response = {
            "답변": raw_answer,
            "후처리": final_answer,
            "문서 일부": context,
            "question_en": query,
            "answer_en": raw_answer,
            "검색_결과": search_results,
            "번역" : translated_answer if language.lower() != "ko" else None
        }

        try:
            # result_path가 없으면 상위 스코프나 기본값으로 설정
            result_path = getattr(self, 'result_path', '../result')
            
            # 알파값으로 폴더 경로 생성
            alpha_str = f"{alpha:.1f}"
            alpha_folder = os.path.join(result_path, f"alpha_{alpha_str}")
            
            # 폴더가 없으면 생성
            os.makedirs(alpha_folder, exist_ok=True)
            
            # 파일 저장 시도
            print(f"저장 시도: {alpha_folder}")  # 디버깅용
            save_response_to_file(
                query=query,
                answer=response["답변"],
                final_answer=response["후처리"] if language.lower() == "ko" else response["번역"],
                context=response["문서 일부"],
                folder=alpha_folder
            )
            print(f"저장 완료: {alpha_folder}")  # 디버깅용
        except Exception as e:
            print(f"결과 저장 중 오류 발생: {e}")
        
        return response
    
    def _load_prompt(self, path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"[ERROR] 프롬프트 로드 실패: {e}")
            raise
    
    def _extract_answer_content(self, text):
        pattern = r"(?:<\|?|<|)?\|?answer\|?(?:\|?>|>)?(.*?)(?:<\|?|<|)?\|?endanswer\|?(?:\|?>|>)?"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else text.strip()
    
    def _remove_chinese_characters(self, text):
        return re.sub(r'[\u4E00-\u9FFF]', '', text)

In [29]:
from model_loader.config import generation_loader
import numpy as np
import time

question_list = [
                "애가 문 못열게 하는거 어케함?", 
                "운전석에서 문 다 잠그는거 할 수 있나?",
                "차키 배터리 뭐사야돼?",
                "하이패스에 빨간불 들어오는데 왜이럼?",
                "시트 너무 낮은데 어떻게 조절함?",
                "창문 올라가다 마는데 이거 왜이래",
                "도난방지인가 그거 어떻게 끄냐?",
                "카드 꽂았는데 하이패스 안되는데 어케함",
                "열선 버튼 누르면 언제 꺼지냐?",
                "스마트키 물에 빠졌는데 써도돼?",
                "하이패스 후불로 하고싶은데 어디다가 말해야대?",
                "차키 안에 두고 내려서 다른 키로 열었는데 안에 있던 키 작동이 안돼.",
                "차키 배터리 어따가 버려야되냐? 일반쓰레기인가?",
                "차 달리다가 자동으로 잠기는거 몇킬로로 달릴때 잠기냐?",
                "지금 차 친구한테 팔려고 하는데 안에 하이패스는 어케 해야되냐",
                "하이패스에서 카드를 확인하십시오 라고 나오는데 뭐가문제야",
                "하이패스 등록하려는데 서류 뭐내야돼?",
                "뒷자리 애가 창문열고 장난치는데 창문 잠그는방법 없어?",
                "임산부는 벨트 매야되나? 위험하지않나?",
                "차 산지 5년 다되가는데 에어백도 점검해야되나?"
                ]
category_list = ['16'] * 20
alpha_values = np.arange(0.0, 1.1, 0.1)

generation_loader = generation_loader
result_base_path = "../result"
qa_system = CarManualQA(generation_loader)

for alpha in alpha_values:
    # 소수점 첫째자리까지 표현하기 위해 포맷팅
    alpha_formatted = round(alpha, 1)
    print(f"\n===== Testing with alpha={alpha_formatted} =====")
    
    for q, c in zip(question_list, category_list):
        print(f"\n새 쿼리: {q}, 카테고리: {c}")
        
        
        # 검색 결과 가져오기 (alpha 값 전달)
        # search_results = test_search_only(q, c, alpha=alpha_formatted)
        # context = "\n\n".join([result["chunk"] for result in search_results])
        
        start_time = time.time()
        # 응답 생성 (alpha 값 전달)
        result = qa_system.generate_response(q, c, alpha=alpha_formatted, language="ko")
        end_time = time.time()
        
        
        
        elapsed_time = end_time - start_time

        print(f"추론시간 : {elapsed_time}")
        print(f"LLM 답변: {result['후처리']}")


===== Testing with alpha=0.0 =====

새 쿼리: 애가 문 못열게 하는거 어케함?, 카테고리: 16
번역 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


번역 중 오류 발생 : Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../model/translate/facebook-m2m100'. Use `repo_type` argument if needed.
저장 시도: ./result/5월9일/gemma3_full/alpha_0.0
저장 완료: ./result/5월9일/gemma3_full/alpha_0.0
추론시간 : 279.2659065723419
LLM 답변: [답변]
18페이지에 따르면, 뒤쪽 도어의 어린이 안전 잠금 장치를 위쪽으로 올려 잠그면 도어는 실내에서 열 수 없습니다. 기능을 해제하려면 실외 열림 레버로 뒤쪽 도어를 열고 어린이 안전 잠금 장치를 해제 방향으로 내리면 됩니다.

[문서]
#### 65페이지
어린이가 도어에 기대거나 측면 에어백 모듈에 가까이 있지 않도록 하십시오.

#### 20페이지
참고
도어의 키 홈이 결빙되어 열리지 않을 경우에는 살짝 두드리거나 키를 뜨겁게 만든 후 여시기 바랍니다.

주의
차량을 주차 또는 정차시키고 떠날때에는 모든 도어를 잠그고 키를 소지하십시오. 그렇게 하지 않으면 차량을 도난당할 수 있습니다.

경고
여름철에 모든 유리창을 닫은 상태에서 어린이, 장애우, 애완동물 등을 차량 안에 두고 절대로 떠나지 마십시오. 차량 실내 온도가 실외보다 더 높게 상승하기 때문에 부상을 입거나 생명을 잃을 수 있습니다.

#### 18페이지
자동도어 잠금해제
차량 충돌 시(감지 센서에 충격 전달시) 도어 잠금을 자동으로 해제시키는기능이 있습니다.
단, 도어 잠금 해제와 관련된 기계적인 장치나 배터리에 문제가 있을 때에는 도어가 잠금

새 쿼리: 운전석에서 문 다 잠그는거 할 수 있나?, 카테고리: 16
번역 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: 